In [103]:
import pandas as pd

data = pd.read_csv('data/dat.csv', sep=';')



In [107]:
data['Образование'] = data['Образование и ВУЗ'].apply(lambda x: ' '.join(x.split(' ')[:2]))

del data['Образование и ВУЗ']

In [111]:
data['Пол'] = data['Пол, возраст'].apply(lambda x: x[:1])
data['Возраст'] = data['Пол, возраст'].apply(lambda x: x.split(' ')[3])

data['Возраст'].astype(int).mean()

del data['Пол, возраст']

In [112]:
data.head()


,ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Обновление резюме,Авто,Образование,Пол,Возраст
0,29000 руб.,Системный администратор,"Советск (Калининградская область) , не готов к...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,16.04.2019 15:59,Имеется собственный автомобиль,Неоконченное высшее,М,39
1,40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Опыт работы 19 лет 5 месяцев Январь 2000 — по...,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",12.04.2019 08:42,Не указано,Высшее образование,М,60
2,20000 руб.,Оператор,"Тверь , не готова к переезду , не готова к ком...",полная занятость,полный день,Опыт работы 10 лет 3 месяца Октябрь 2004 — Де...,ПАО Сбербанк,Кассир-операционист,16.04.2019 08:35,Не указано,Среднее специальное,Ж,36
3,100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",Опыт работы 18 лет 9 месяцев Август 2017 — Ап...,OpenSoft,Инженер-программист,08.04.2019 14:23,Не указано,Высшее образование,М,38
4,140000 руб.,Региональный менеджер по продажам,"Москва , не готова к переезду , готова к коман...",полная занятость,полный день,Опыт работы 5 лет 7 месяцев Региональный мене...,Мармелад,Менеджер по продажам,22.04.2019 10:32,Не указано,Высшее образование,Ж,26


In [241]:
import numpy as np

def trans(arg):
    if arg is np.nan:
        return np.nan
    if arg == 'Не указано':
        return np.nan
    month_id = ['месяцев', 'месяца', 'месяц']
    year_id = ['лет', 'год', 'года']
    splitted = str(arg).split(' ')[2:]
    if splitted[1] in year_id:
        mon = int(splitted[0]) * 12
        if splitted[3] in month_id:
            totmon = int(splitted[2]) + mon
            return totmon
        else: return mon
    if splitted[1] in month_id:
        mon = int(splitted[0])
        return mon
    else: pass
data['Опыт работы (месяц)'] = data['Опыт работы'].apply(trans)



In [194]:
def cityloc(arg):
    million_cities = ['Новосибирск', 'Екатеринбург', 'Нижний Новгород', 'Казань',
                      'Челябинск', 'Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск',
                      'Пермь', 'Воронеж', 'Волгоград' ]
    splitted = str(arg).split(' ')

    if splitted[0] == 'Санкт-Петербург' or splitted[0] == 'Москва':
        return splitted[0]
    elif splitted[0] in million_cities:
        return 'город-милионник'
    else:
        return 'Другие'

    
    
def citymove(arg):
    splitted = str(arg).split(',')
    dbsplit = splitted[-2].split(' ')
    if 'не' in dbsplit:
        return False
    else:
        return True
 

def comand(arg):
    splitted = str(arg).split(',')
    dbsplit = splitted[-1].split(' ')
    if 'не' in dbsplit:
        return False
    else:
        return True
  
data['Город'] = data['Город, переезд, командировки'].apply(cityloc)
print(data['Город'].value_counts())

data['Готовность к переезду'] = data['Город, переезд, командировки'].apply(citymove)
data['Готовность к переезду'].value_counts()

data['Готовность к командировкам'] = data['Город, переезд, командировки'].apply(comand)
data['Готовность к командировкам'].value_counts()




Москва             16621
Другие             16452
город-милионник     6734
Санкт-Петербург     4937
Name: Город, dtype: int64


True     31716
False    13028
Name: Готовность к командировкам, dtype: int64

In [199]:
mask = (data['Готовность к переезду'] == True) & (data['Готовность к командировкам'] == True)
data[mask].count()
del data['Город, переезд, командировки']


In [204]:
data.head()

,ЗП,Ищет работу на должность:,Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Обновление резюме,Авто,Образование,Пол,Возраст,Город,Готовность к переезду,Готовность к командировкам
0,29000 руб.,Системный администратор,"частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,16.04.2019 15:59,Имеется собственный автомобиль,Неоконченное высшее,М,39,Другие,False,False
1,40000 руб.,Технический писатель,"частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Опыт работы 19 лет 5 месяцев Январь 2000 — по...,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",12.04.2019 08:42,Не указано,Высшее образование,М,60,Другие,False,True
2,20000 руб.,Оператор,полная занятость,полный день,Опыт работы 10 лет 3 месяца Октябрь 2004 — Де...,ПАО Сбербанк,Кассир-операционист,16.04.2019 08:35,Не указано,Среднее специальное,Ж,36,Другие,False,False
3,100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,"частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",Опыт работы 18 лет 9 месяцев Август 2017 — Ап...,OpenSoft,Инженер-программист,08.04.2019 14:23,Не указано,Высшее образование,М,38,Другие,False,True
4,140000 руб.,Региональный менеджер по продажам,полная занятость,полный день,Опыт работы 5 лет 7 месяцев Региональный мене...,Мармелад,Менеджер по продажам,22.04.2019 10:32,Не указано,Высшее образование,Ж,26,Москва,False,True


In [226]:
busyTypes = ['полная занятость', 'частичная занятость', 'проектная работа', 'стажировка', 'волонтерство']
data['полная занятость'] = data['Занятость'].apply(lambda x: True if ('полная занятость' in x) else False)

data['частичная занятость'] = data['Занятость'].apply(lambda x: True if ('частичная занятость' in x) else False)

data['проектная работа'] = data['Занятость'].apply(lambda x: True if ('проектная работа' in x) else False)

data['стажировка'] = data['Занятость'].apply(lambda x: True if ('стажировка' in x) else False)

data['волонтерство'] = data['Занятость'].apply(lambda x: True if ('волонтерство' in x) else False)

msk = (data['проектная работа'] == True) & (data['волонтерство'] == True)
data[msk].shape
    

(436, 25)

In [224]:
data['График'][0]

'гибкий график, полный день, сменный график, вахтовый метод, удаленная работа'

In [225]:
data['гибкий график'] = data['График'].apply(lambda x: True if ('гибкий график' in x) else False)

data['полный день'] = data['График'].apply(lambda x: True if ('полный день' in x) else False)

data['сменный график'] = data['График'].apply(lambda x: True if ('сменный график' in x) else False)

data['удаленная работа'] = data['График'].apply(lambda x: True if ('удаленная работа' in x) else False)

data['вахтовый метод'] = data['График'].apply(lambda x: True if ('вахтовый метод' in x) else False)

msk2 = (data['вахтовый метод'] == True) & (data['гибкий график'] == True)
data[msk2].shape




(2311, 25)

In [227]:
exrates = pd.read_csv('data/ExchangeRates.csv')
exrates

,currency,per,date,time,close,vol,proportion
0,USD,D,29/12/17,00:00,57.6291,0,1
1,USD,D,30/12/17,00:00,57.6002,0,1
2,USD,D,31/12/17,00:00,57.6002,0,1
3,USD,D,01/01/18,00:00,57.6002,0,1
4,USD,D,02/01/18,00:00,57.6002,0,1
...,...,...,...,...,...,...,...
5659,AZN,D,02/12/19,00:00,37.7729,0,1
5660,AZN,D,03/12/19,00:00,37.9662,0,1
5661,AZN,D,04/12/19,00:00,37.8073,0,1
5662,AZN,D,05/12/19,00:00,37.8396,0,1


In [239]:
def get_salary_num(arg):
    salary = float(arg.split(' ')[0])
    return salary

def get_salary_currency(arg):
    currency_dict = {
        'USD': 'USD', 'KZT': 'KZT',
        'грн': 'UAH', 'белруб': 'BYN',
        'EUR': 'EUR', 'KGS': 'KGS',
        'сум': 'UZS', 'AZN': 'AZN'
    }
    curr = arg.split(' ')[1].replace('.', '')
    if curr == 'руб':
        return 'RUB'
    else:
        return currency_dict[curr]
    
    
exrates['date'] = pd.to_datetime(exrates['date']).dt.date
data['Обновление резюме'] = pd.to_datetime(data['Обновление резюме']).dt.date
data['ЗП (tmp)'] = data['ЗП'].apply(get_salary_num)
data['Курс (tmp)'] = data['ЗП'].apply(get_salary_currency)


merged = data.merge(exrates, left_on=['Курс (tmp)', 'Обновление резюме'], right_on=['currency', 'date'], how='left')

merged['close'] = merged['close'].fillna(1)
merged['proportion'] = merged['proportion'].fillna(1)

data['ЗП (руб)'] = merged['close'] * merged['ЗП (tmp)'] / merged['proportion']
data = data.drop(['ЗП', 'ЗП (tmp)', 'Курс (tmp)'], axis=1)

print(round(data['ЗП (руб)'].median()/1000))



59


In [255]:
data.info()

data.to_csv('data/nd2.csv')
    
    


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44744 entries, 0 to 44743
Data columns (total 23 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Ищет работу на должность:        44744 non-null  object 
 1   Последнее/нынешнее место работы  44743 non-null  object 
 2   Последняя/нынешняя должность     44742 non-null  object 
 3   Обновление резюме                44744 non-null  object 
 4   Авто                             44744 non-null  object 
 5   Образование                      44744 non-null  object 
 6   Пол                              44744 non-null  object 
 7   Возраст                          44744 non-null  object 
 8   Город                            44744 non-null  object 
 9   Готовность к переезду            44744 non-null  bool   
 10  Готовность к командировкам       44744 non-null  bool   
 11  полная занятость                 44744 non-null  bool   
 12  частичная занятост